<a href="https://colab.research.google.com/github/yulin6666/996.ICU/blob/master/cat_dog_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import numpy as np
import os, math,cv2
import tensorflow as tf
from IPython.display import display, Image, HTML

# 训练集
TRAIN_DIR='./drive/My Drive/Colab Notebooks/cat_dog_data/data/train/'
train_images_all = [TRAIN_DIR + i for i in os.listdir(TRAIN_DIR)]
train_images_dog = [i for i in train_images_all if 'dog' in i]
train_images_cat = [i for i in train_images_all if 'cat' in i]
train_images_filepaths = train_images_dog + train_images_cat
TRAIN_DATA_NUM = 2000 # 狗和猫各自的训练图片数量
VALIDATION_DATA_NUM = 500 # 狗和猫各自的验证图片数量
TEST_NUM = 1000 # 测试集数量
TRAIN_NUM = TRAIN_DATA_NUM * 2 # 训练集的图片数量，不包含验证集
VALIDATION_NUM = VALIDATION_DATA_NUM * 2 # 验证集的图片数量
train_images_filepaths = train_images_filepaths[:TRAIN_NUM+VALIDATION_NUM]
train_images_labels = np.array([1] * (TRAIN_DATA_NUM + VALIDATION_DATA_NUM) \
                               + [0] * (TRAIN_DATA_NUM + VALIDATION_DATA_NUM)) # 1表示dog，0表示cat
print("train num:",np.array(train_images_filepaths).shape)

# 测试集
TEST_DIR = './drive/My Drive/Colab Notebooks/cat_dog_data/data/test/'
test_images_filepaths = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
test_images_filepaths = test_images_filepaths[:TEST_NUM]
print("test num:",np.array(test_images_filepaths).shape)

#预处理图片
def read_image(file_path):
    img = cv2.imread(file_path,cv2.IMREAD_COLOR)
    if img.shape[0] > img.shape[1]:
        img_size = (IMAGE_SIZE, int(round(float(img.shape[1] / img.shape[0]) * IMAGE_SIZE)))
    else:
        img_size = (int(round(float(img.shape[0] / img.shape[1]) * IMAGE_SIZE)), IMAGE_SIZE)  
    tmp_img = cv2.resize(img, (img_size[1], img_size[0]), interpolation=cv2.INTER_CUBIC)
    ret_img = cv2.copyMakeBorder(tmp_img, 0, IMAGE_SIZE-img_size[0], 0, IMAGE_SIZE-img_size[1], cv2.BORDER_CONSTANT, 0)
    return ret_img[:,:,::-1]
    
IMAGE_SIZE = 250
CHANNELS = 3
train_dataset = np.ndarray((TRAIN_NUM + VALIDATION_NUM, IMAGE_SIZE, IMAGE_SIZE, CHANNELS), dtype=np.uint8)
for i, image_path in enumerate(train_images_filepaths):
    if(i%1000==0):
      print("processed train image:",i)
    train_dataset[i]=read_image(image_path)
print("Train shape: {}".format(train_dataset.shape))

# 载入测试集
test_dataset = np.ndarray((TEST_NUM, IMAGE_SIZE, IMAGE_SIZE, CHANNELS), dtype=np.uint8)
for i, image_path in enumerate(test_images_filepaths):
    if(i%1000==0):
      print("processed test image:",i)
    test_dataset[i] = read_image(image_path)
print("Test shape:",test_dataset.shape)

#随机打乱数据
np.random.seed(201712)
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:,:,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

train_dataset_rand, train_labels_rand = randomize(train_dataset,train_images_labels)
train = train_dataset_rand[:TRAIN_NUM,:,:,:]
train_labels = train_labels_rand[:TRAIN_NUM]
valid = train_dataset_rand[-VALIDATION_NUM:]
valid_labels = train_labels_rand[-VALIDATION_NUM:]
print('Train shape: ', train.shape, train_labels.shape)
print('Valid shape: ', valid.shape, valid_labels.shape)


#生成字符串型的属性
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 生成整数型的属性
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 把数据写到多个文件中
def save_multiple_tfrecords(images, labels, filename, instances_per_file):
    image_num = labels.shape[0]
    file_num = math.ceil(image_num / instances_per_file) # 计算需要写的文件总数
    
    # 遍历每一张图片
    write_num = instances_per_file
    cur_file_no = -1
    for i in range(image_num):
        # 如果一个文件的图片达到预定的数目，则创建新的文件继续写
        if write_num == instances_per_file:
            write_num = 0
            cur_file_no += 1
            write_filename = filename + '.tfrecords-%.4d-of-%.4d' % (cur_file_no, file_num)
            print('Writing ' + write_filename)
            writer = tf.python_io.TFRecordWriter(write_filename)
        # 写图片到文件
        image_bytes = images[i].tostring()
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    'label': _int64_feature(labels[i]),
                    'image_raw': _bytes_feature(image_bytes)
            }))
        writer.write(example.SerializeToString())
        write_num += 1
    writer.close()
    print('Writing End.')

    
save_multiple_tfrecords(train, train_labels, './drive/My Drive/Colab Notebooks/cat_dog_data/data/tfrecord/train', 1024)
save_multiple_tfrecords(valid, valid_labels, './drive/My Drive/Colab Notebooks/cat_dog_data/data/tfrecord/valid', VALIDATION_NUM)
save_multiple_tfrecords(test_dataset, np.array([0] * TEST_NUM), './drive/My Drive/Colab Notebooks/cat_dog_data/data/tfrecord/test', TEST_NUM)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
train num: (5000,)
test num: (1000,)
processed train image: 0
processed train image: 1000
processed train image: 2000
processed train image: 3000
processed train image: 4000
Train shape: (5000, 250, 250, 3)
processed test image: 0
Test shape: (1000, 250, 250, 3)
Train shape:  (4000, 250, 250, 3) (4000,)
Valid shape:  (1000, 250, 250, 3) (1000,)
Writing ./drive/My Drive/Colab Notebooks/cat_dog_data/data/tfrecord/train.tfrecords-0000-of-0004
Writing ./drive/My Drive/Colab Notebooks/cat_dog_data/data/tfrecord/train.tfrecords-0001-of-0004
Writing ./drive/My Drive/Colab Notebooks/cat_dog_data/data/tfrecord/train.tfrecords-0002-of-0004
Writing ./drive/My Drive/Colab Notebooks/cat_dog_data/data/tfrecord/train.tfrecords-0003-of-0004
Writing End.
Writing ./drive/My Drive/Colab Notebooks/cat_dog_data/data/tfrecord/valid.tfrecords-0000-of-0001
Writing End.
Writing ./dri